In [1]:
import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import pandas as pd  
import numpy as np  

import tools.Sample_Tools as smpl
# import tools.Pretreat_Tools as pretreat
from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)

from base.JuUnits import parallal_task,task_chunk_split
from base.JuUnits import excute_for_multidates


# import inspect
import ind.Ind_Turn_Amihud_Pvc_2023 as new2023

%load_ext autoreload
%autoreload 2
%aimport ind.Ind_Turn_Amihud_Pvc_2023

In [4]:
# #读取已经缓存的复权数据，并预处理
tail = True
origin_start = None
if tail:
    df_tail = load_cache('all_tail_qfq',cache_type=CACHE_TYPE.STOCK).sort_index()
    df_train = load_cache('all_train_qfq',cache_type=CACHE_TYPE.STOCK).sort_index()
    origin_start = df_tail.index.get_level_values(0).min()
    year_data_reserve = origin_start.year - 2

    df_all = pd.concat([df_train.loc[df_train.index.get_level_values(0) >= str(year_data_reserve)],df_tail],axis=0)
else:
    df_all = load_cache('all_train_qfq',cache_type=CACHE_TYPE.STOCK).sort_index()
    
smpl.optimize_data_type(df_all)

df_tail = None
df_train = None

stock_df = df_all

In [5]:
#############测试#######################
# ['000001','000008','600600','000729']
# stock_df___ = stock_df.loc[(slice(None),stock_df.index.get_level_values(1).unique().to_list()[0:8]),:]

stock_codes = stock_df.index.get_level_values(1).unique().to_list()
np.random.shuffle(stock_codes)

def generate_factors(codes, data_df=None):
    import ind.Ind_Turn_Amihud_Pvc_2023 as new2023
    from base.JuUnits import excute_for_multidates
    import numpy as np
    import pandas as pd

    df = data_df.loc[(slice(None),codes),:]
    cur_ret = new2023.get_cur_ret(df)
    
    turn_cv = new2023.get_turn_cv(df)
    amihud = new2023.get_amihud(df, cur_ret)
    amihud_adv = new2023.get_amihud_adv(df, cur_ret)
    amihud_negative = new2023.get_amihud_negative(df)
    amihud_cv = new2023.get_amihud_cv(df, cur_ret)
    obv = new2023.get_obv(df, cur_ret)
    pvc = new2023.get_pvc(df)
    pvc_optimize = new2023.get_pvc_optimize(df,cur_ret,pvc)
    factor = pd.concat([turn_cv,
                        amihud, amihud_adv, amihud_negative, amihud_cv,
                        obv,pvc,pvc_optimize
                       ],axis=1).sort_index()
    return factor

worker=6
task = task_chunk_split(stock_codes, worker)


results = parallal_task(worker, generate_factors, task, data_df=stock_df)
results = pd.concat(results).sort_index()

if tail:
    save_cache('turn_Amihud_Pvc_2023_tail', results, cache_type=CACHE_TYPE.FACTOR)
else:
    save_cache('turn_Amihud_Pvc_2023_train', results, cache_type=CACHE_TYPE.FACTOR)

Now in the main code. Process name is: base.JuUnits
base.JuUnits, subpid:4080  pid:9696


  0%|          | 0/6 [00:00<?, ?it/s]

In [8]:
# x = load_cache('turn_Amihud_Pvc_2023_tail', cache_type=CACHE_TYPE.FACTOR)
x.iloc[1097124 :2597124].corr()

,turn_cv,amihud,amihud_adv,amihud_negative,amihud_cv,obv,pvc,pvc_optimize
turn_cv,1.000000,0.021780,0.045671,0.057625,-0.183143,0.032684,0.252184,0.161284
amihud,0.021780,1.000000,0.372477,0.236401,-0.032716,-0.003067,-0.037934,-0.024718
amihud_adv,0.045671,0.372477,1.000000,0.626136,-0.063919,-0.006351,-0.084971,-0.058102
amihud_negative,0.057625,0.236401,0.626136,1.000000,-0.054422,-0.010953,-0.101625,-0.064020
amihud_cv,-0.183143,-0.032716,-0.063919,-0.054422,1.000000,-0.028804,-0.138146,-0.101259
obv,0.032684,-0.003067,-0.006351,-0.010953,-0.028804,1.000000,0.050058,0.002613
pvc,0.252184,-0.037934,-0.084971,-0.101625,-0.138146,0.050058,1.000000,0.943885
pvc_optimize,0.161284,-0.024718,-0.058102,-0.064020,-0.101259,0.002613,0.943885,1.000000
